In [1]:
# Connect to google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#!unzip "/content/drive/MyDrive/Final_Project/Train_dataset_small.zip" -d "/content/drive/MyDrive/Final_Project/dataset2"

In [3]:
import torch
from torchvision import datasets, transforms
from torchvision.transforms import v2
from torch.utils.data import DataLoader, Dataset, Subset, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as  np
from torchvision import utils
import matplotlib.pyplot as plt
import os

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
path='/content/drive/MyDrive/Final_Project/dataset2/Train_dataset_new'
n_channels = 3
n_classes = 15
batch_size = 64
learning_rate = 0.001
transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

In [6]:
dataset = datasets.ImageFolder(root=path, transform=transform)

In [7]:
# Split the dataset into train, test and validation.
total_dataset_len = len(dataset)
train_dataset_len = int(0.7 * total_dataset_len)
val_dataset_len = int(0.15 * total_dataset_len)
test_dataset_len = total_dataset_len - train_dataset_len - val_dataset_len
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_dataset_len, val_dataset_len, test_dataset_len])

In [8]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, num_workers=2, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, num_workers=2, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, num_workers=2, shuffle=False)
print(len(train_loader))

532


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class VGG_Places365(nn.Module):
    def __init__(self):
        super(VGG_Places365, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        init.normal_(self.conv1.weight, mean=0.0, std=0.01)
        init.constant_(self.conv1.bias, 0.0)

        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        init.normal_(self.conv2.weight, mean=0.0, std=0.01)
        init.constant_(self.conv2.bias, 0.0)

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        init.normal_(self.conv3.weight, mean=0.0, std=0.01)
        init.constant_(self.conv3.bias, 0.0)

        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        init.normal_(self.conv4.weight, mean=0.0, std=0.01)
        init.constant_(self.conv4.bias, 0.0)

        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        init.normal_(self.conv5.weight, mean=0.0, std=0.01)
        init.constant_(self.conv5.bias, 0.0)

        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        init.normal_(self.conv6.weight, mean=0.0, std=0.01)
        init.constant_(self.conv6.bias, 0.0)

        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        init.normal_(self.conv7.weight, mean=0.0, std=0.01)
        init.constant_(self.conv7.bias, 0.0)

        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv8 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        init.normal_(self.conv8.weight, mean=0.0, std=0.01)
        init.constant_(self.conv8.bias, 0.0)

        self.conv9 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        init.normal_(self.conv9.weight, mean=0.0, std=0.01)
        init.constant_(self.conv9.bias, 0.0)

        self.conv10 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        init.normal_(self.conv10.weight, mean=0.0, std=0.01)
        init.constant_(self.conv10.bias, 0.0)

        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv11 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        init.normal_(self.conv11.weight, mean=0.0, std=0.01)
        init.constant_(self.conv11.bias, 0.0)

        self.conv12 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        init.normal_(self.conv12.weight, mean=0.0, std=0.01)
        init.constant_(self.conv12.bias, 0.0)

        self.conv13 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        init.normal_(self.conv13.weight, mean=0.0, std=0.01)
        init.constant_(self.conv13.bias, 0.0)

        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc6 = nn.Linear(512 * 7 * 7, 4096)
        init.normal_(self.fc6.weight, mean=0.0, std=0.01)
        init.constant_(self.fc6.bias, 0.0)

        self.fc7 = nn.Linear(4096, 4096)
        init.normal_(self.fc7.weight, mean=0.0, std=0.01)
        init.constant_(self.fc7.bias, 0.0)

        self.fc8 = nn.Linear(4096, 365)

        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)

        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.pool3(x)

        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        x = self.pool4(x)

        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = self.pool5(x)

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc6(x))
        x = self.dropout(x)
        x = F.relu(self.fc7(x))
        x = self.dropout(x)
        x = self.fc8(x)

        x = F.softmax(x, dim=1)

        return x

# Create an instance of the model
model = VGG_Places365()
model.to(device)

# Print the model architecture
print(model)


VGG_Places365(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv8): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv9): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv10): Conv2d(512

In [11]:
def train_model(model, train_loader, val_loader, criterion, optimizer, n_epochs):
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss = train_loss / len(train_loader)
        train_acc = train_correct / train_total
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = (val_correct / val_total) * 100
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f'Epoch {epoch+1}/{n_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    return train_losses, train_accs, val_losses, val_accs

In [23]:
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), lr = 0.01,
                                momentum=0.9,
                                weight_decay=0.0005)


In [24]:
train_losses, train_accs, val_losses, val_accs = train_model(model, train_loader, val_loader, criterion, optimizer, 2)

Epoch 1/2:
Train Loss: 5.8999, Train Acc: 0.0896
Val Loss: 5.8999, Val Acc: 0.1044
Epoch 2/2:
Train Loss: 5.8998, Train Acc: 0.1012
Val Loss: 5.8998, Val Acc: 0.1044


In [14]:
!pip install git+https://github.com/BVLC/caffe.git@master

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [15]:
import caffe

# Load the Caffe model
caffe.set_mode_cpu()
model_def = '/content/drive/MyDrive/Final_Project/vgg.prototxt'
net = caffe.Net(model_def)


ModuleNotFoundError: No module named 'caffe'

In [16]:
!python -m venv my_caffe_env
!source my_caffe_env/bin/activate
!pip install git+https://github.com/BVLC/caffe.git@master
import caffe

The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.10-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: /content/my_caffe_env/bin/python3

/bin/bash: line 1: my_caffe_env/bin/activate: No such file or directory
  Cloning https://github.com/BVLC/caffe.git (to revision master) to /tmp/pip-req-build-qnv2g7v7
  Running command git clone --filter=blob:none --quiet https://github.com/BVLC/caffe.git /tmp/pip-req-build-qnv2g7v7
  Resolved https://github.com/BVLC/caffe.git to commit 9b891540183ddc834a02b2bd81b31afae71b2153
ERROR: git+https://github.com/BVLC/caffe.git@master does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


ModuleNotFoundError: No module named 'caffe'